<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update BDD Growth

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook allows to send data to Google Sheets to a Google Sheets spreadsheet.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
try:
    import openai
except:
    !pip install openai --user
    import openai
import time

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_reactions`: Name of the file with reactions to be retrieved.
- `file_comments`: Name of the file with comments to be retrieved.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_name = "linkedin_interactions"
openai_api_key = naas.secret.get("OPENAI_API_KEY")

# Outputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "GROWTH"
append = False

## Model

### Get BDD interactions

In [ ]:
df_interactions = pload(input_dir, file_name)    
print('🗂️ Interactions:', len(df_interactions))
df_interactions.head(1)

### Get data "GROWTH" from Google Sheets spreadsheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

### Get data "REF_CONTACT" from Google Sheets spreadsheet

In [ ]:
ref_contact = gsheet.connect(spreadsheet_url).get(sheet_name="REF_CONTACT")
print("Rows:", len(ref_contact))
ref_contact.head(5)

### Get data "REF_COMPANY" from Google Sheets spreadsheet

In [ ]:
ref_company = gsheet.connect(spreadsheet_url).get(sheet_name="REF_COMPANY")
print("Rows:", len(ref_company))
ref_company.head(1)

### Create Interest data

In [ ]:
df_interests = df_interactions.copy()
to_select = [
    "PROFILE_URL",
    "CONTENT_TITLE",
    "CONTENT_URL",
    "INTERACTION",
    "INTERACTION_CONTENT"
]
df_interests = df_interests[to_select].sort_values(by="PROFILE_URL").reset_index(drop=True)

# Create interest data
profiles = df_interests["PROFILE_URL"].unique()
interests_data = {}
for profile in profiles:
    tmp_df = df_interests[df_interests["PROFILE_URL"] == profile][:5].reset_index(drop=True)
    interests = ""
    for row in tmp_df.itertuples():
        content_title = row.CONTENT_TITLE
        content_url = row.CONTENT_URL
        interaction = row.INTERACTION
        interaction_content = row.INTERACTION_CONTENT
        if interaction == "POST_REACTION":
            interest = f"{interests}{interaction_content.capitalize()} '{content_title.strip()}' ({content_url}),"
        else:
            interest = f"{interests} Comment '{interaction_content}' on '{content_title.strip()}' ({content_url}),"
        
        interests = f"{interests}{interest}"
    interests_data[profile] = interests.strip()

### Final dataframe

In [ ]:
df = df_interactions.copy()

# Map interest
df["INTERESTS"] = df["PROFILE_URL"].map(interests_data)

# Groupby
to_group = [
    "ENTITY",
    "SCENARIO",
    "PLATFORM",
    "FULLNAME",
    "OCCUPATION",
    "PROFILE_URL",
    "INTERESTS",
]
to_agg = {
    "INTERACTION_SCORE": "sum"
}
df = df.groupby(to_group, as_index=False).agg(to_agg).sort_values(by=["SCENARIO", "INTERACTION_SCORE"], ascending=[False, False]).reset_index(drop=True)
print("Rows:", len(df))
df.head(5)

## Output

### Send data to Google Sheets spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name, data=df, append=append)